<a href="https://colab.research.google.com/github/Jaswanthk07/Jaswanthk07/blob/main/webprojectapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **CREATING THE MODEL**

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [4]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_epochs = 25
batch_size = 128
learning_rate = 0.001

In [5]:
# Data augmentation and normalization for training
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Normalization for validation/test
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


In [6]:
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=train_transform)
val_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=val_transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


100%|██████████| 170498071/170498071 [00:02<00:00, 70847566.45it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
# Define the model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)


In [8]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=5, verbose=True)

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [9]:
# Function to calculate accuracy
def accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    return correct / labels.size(0)


In [10]:
# Training the model
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
        train_acc += accuracy(outputs, labels) * inputs.size(0)
        # Average loss and accuracy
    train_loss = train_loss / len(train_loader.dataset)
    train_acc = train_acc / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            val_acc += accuracy(outputs, labels) * inputs.size(0)

    # Average validation loss and accuracy
    val_loss = val_loss / len(val_loader.dataset)
    val_acc = val_acc / len(val_loader.dataset)

    # Adjust learning rate based on validation accuracy
    scheduler.step(val_acc)

    # Print training progress
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

print('Finished Training')

Epoch [1/25], Train Loss: 1.5446, Train Acc: 0.4366, Val Loss: 1.2082, Val Acc: 0.5680
Epoch [2/25], Train Loss: 1.1933, Train Acc: 0.5705, Val Loss: 0.9950, Val Acc: 0.6465
Epoch [3/25], Train Loss: 1.0464, Train Acc: 0.6254, Val Loss: 0.9209, Val Acc: 0.6789
Epoch [4/25], Train Loss: 0.9439, Train Acc: 0.6665, Val Loss: 0.8329, Val Acc: 0.7062
Epoch [5/25], Train Loss: 0.8785, Train Acc: 0.6901, Val Loss: 0.7916, Val Acc: 0.7303
Epoch [6/25], Train Loss: 0.8258, Train Acc: 0.7109, Val Loss: 0.7936, Val Acc: 0.7297
Epoch [7/25], Train Loss: 0.7862, Train Acc: 0.7234, Val Loss: 0.7431, Val Acc: 0.7423
Epoch [8/25], Train Loss: 0.7489, Train Acc: 0.7382, Val Loss: 0.7078, Val Acc: 0.7547
Epoch [9/25], Train Loss: 0.7120, Train Acc: 0.7494, Val Loss: 0.6896, Val Acc: 0.7595
Epoch [10/25], Train Loss: 0.6904, Train Acc: 0.7579, Val Loss: 0.6929, Val Acc: 0.7647
Epoch [11/25], Train Loss: 0.6658, Train Acc: 0.7676, Val Loss: 0.6706, Val Acc: 0.7727
Epoch [12/25], Train Loss: 0.6498, Train 

## **TESTING THE MODEL ON A SAMPLE IMAGE**

In [11]:
# Load the image
from PIL import Image
img_path = "/content/images/image.jpg"
image = Image.open(img_path)

In [12]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])  # Example normalization
])

# Apply the transformation
normalized_tensor_image = transform(image)

# Verify the size of the normalized tensor
print(normalized_tensor_image.size())  # Should print torch.Size([3, 32, 32])


torch.Size([3, 32, 32])


In [13]:
model.eval()
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
with torch.no_grad():
    op = model(normalized_tensor_image)
    _, predicted = torch.max(op, 1)

# Print the predicted class
print(f'Predicted class: {classes[predicted.item()]}')

Predicted class: horse


In [14]:
# Classify the image and get probabilities
with torch.no_grad():
    outputs = model(normalized_tensor_image)
    probabilities = F.softmax(outputs, dim=1)  # Apply softmax to get probabilities

# Print the probabilities for each class
probabilities = probabilities.cpu().numpy().flatten()  # Move to CPU and flatten the tensor
for i, prob in enumerate(probabilities):
    print(f' {classes[i]}: {prob:.4f}')

 plane: 0.0009
 car: 0.0008
 bird: 0.0000
 cat: 0.1326
 deer: 0.0002
 dog: 0.1197
 frog: 0.0000
 horse: 0.7353
 ship: 0.0000
 truck: 0.0104


## **app.py**

In [15]:
%%writefile app.py
import streamlit as st
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

# Ensure you have the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model
class SimpleCNN(torch.nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load the trained model weights
model = SimpleCNN().to(device)
model.load_state_dict(torch.load('cifar10_model.pth', map_location=device))
model.eval()

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define the classes
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

def predict(image):
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)
    return classes[predicted.item()]

# Streamlit app
st.title("Image Classification with CIFAR-10")
st.write("Upload an image to classify")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])
if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image.', use_column_width=True)
    st.write("")
    st.write("Classifying...")
    label = predict(image)
    st.write(f"The uploaded image is classified as: {label}")

Writing app.py


## **WEB APP RELATED**

In [16]:
# save
torch.save(model.state_dict(), 'cifar10_model.pth')

In [17]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00


In [37]:
!wget -q -O - ipv4.icanhazip.com

104.196.220.62


In [ ]:
!streamlit run app.py &>/dev/null & npx localtunnel --port 8501

npx: installed 22 in 3.441s
your url is: https://tasty-buckets-drive.loca.lt


## **DEPLOYMENT**

In [41]:
!pip freeze > requirements.txt

In [42]:
from google.colab import files
files.download('requirements.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [43]:
required_packages = ["streamlit", "torch", "torchvision"]
with open('requirements.txt', 'w') as f:
    for package in required_packages:
        f.write(package + "\n")
